### Inception Network (GoogleNet)

- 여러 사이즈의 커널들을 한꺼번에 결합하는 방식을 사용하며, 이를 묶어서 inception module이라고 한다.
- 여러 개의 inception module을 연속적으로 이어서 구성하고 여러 사이즈의 필터들이 서로 다른 공간 기반으로 feature들을 추출한다.
- inception module을 결합하면서 보다 풍부한 Feature Extractor Layer를 구성하게 된다.
- 하지만 여러 사이즈의 커널을 결합하게 되면, Convolution 연산을 수행할 때 파라미터 수가 증가되고 과적합으로 이어진다.
- 이를 극복하고자 연산을 수행하기 전에 1X1 Convolution을 적용해서 파라미터 수를 획기적으로 감소시킨다.
- 1X1 Convolution을 적용하면 입력 데이터의 특징을 함축적으로 표현하면서 파라미터 수를 줄이는 차원 축소 역할을 수행하게 된다.  
<sub>*채널수만 변경할 수 있다.  
*파라미터 수가 늘어나는 것을 방지하며 층은 깊게 쌓는다(차원축소 역할이 있고 불필요한 파라미터를 차단함).
*한가지 방식에 치우치지 않으며 다양한 부분을 볼 수 있다.  
특히 학습 Dense 부분이 없는데, 이미 층을 많이 쌓아 처리를 했기 때문에 아래에서 바로 분류한다.</sub>

<img src="./images/GoogLeNet.png" width="550" style="margin-top:20px; margin-left: 0">
<img src="./images/Inception_Network.png" width="900" style="margin-top:20px; margin-left: 0">

#### 1X1 Convolution
- 행과 열의 크기 변환 없이 Channel의 수를 조절할 수 있고, weight 및 비선형성을 추가하는 역할을 한다.
- 행과 열의 사이즈를 줄이고 싶다면, Pooling을 사용하면 되고, 채널 수만 줄이고 싶다면 1X1 Convolution을 사용하면 된다.

<img src="./images/1x1.png" width="800">

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler

from tensorflow.keras.layers import Concatenate

def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, 
                     filters_pool_reduce, name=None):
   
    # 첫번째 1x1 Conv
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)
    
    # 3x3 적용 전 1x1 conv
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)
    
    # 5x5 적용 전 1x1 Conv
    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)

    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool = Conv2D(filters_pool_reduce, (1, 1), padding='same', activation='relu')(pool)
    
    # 1x1 결과, 3x3 결과, 5x5 결과, pool이후 1x1 결과 feature map을 채널(axis=-1) 기준으로 Concat 적용. 
    # Concatenate는 사이즈는 그대로이고, 각 채널 수를 더한다. 즉, 그대로 뒤에 연결된다.
    output = Concatenate(axis=-1, name=name)([conv_1x1, conv_3x3, conv_5x5, pool])
    return output

In [2]:
def googlenet(in_shape=(224, 224, 3), n_classes=10):
    input_tensor = Input(in_shape)

    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7_2')(input_tensor)
    x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3_2')(x)
    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3_1')(x)
    x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3_1')(x)
    x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3_2')(x)
    
    # 첫번째 inception 모듈
    x = inception_module(x, filters_1x1=64,
                         filters_3x3_reduce=96,
                         filters_3x3=128,
                         filters_5x5_reduce=16,
                         filters_5x5=32,
                         filters_pool_reduce=32,
                         name='inception_3a')
    # 두번째 inception 모듈
    x = inception_module(x,
                         filters_1x1=128,
                         filters_3x3_reduce=128,
                         filters_3x3=192,
                         filters_5x5_reduce=32,
                         filters_5x5=96,
                         filters_pool_reduce=64,
                         name='inception_3b')

    x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3_2')(x)
    
    # 세번째 inception 모듈
    x = inception_module(x,
                         filters_1x1=192,
                         filters_3x3_reduce=96,
                         filters_3x3=208,
                         filters_5x5_reduce=16,
                         filters_5x5=48,
                         filters_pool_reduce=64,
                         name='inception_4a')
    # 네번째 inception 모듈
    x = inception_module(x,
                         filters_1x1=160,
                         filters_3x3_reduce=112,
                         filters_3x3=224,
                         filters_5x5_reduce=24,
                         filters_5x5=64,
                         filters_pool_reduce=64,
                         name='inception_4b')
    
    # 다섯번째 inception 모듈
    x = inception_module(x,
                         filters_1x1=128,
                         filters_3x3_reduce=128,
                         filters_3x3=256,
                         filters_5x5_reduce=24,
                         filters_5x5=64,
                         filters_pool_reduce=64,
                         name='inception_4c')
    # 여섯번째 inception 모듈
    x = inception_module(x,
                         filters_1x1=112,
                         filters_3x3_reduce=144,
                         filters_3x3=288,
                         filters_5x5_reduce=32,
                         filters_5x5=64,
                         filters_pool_reduce=64,
                         name='inception_4d')
    # 일곱번째 inception 모듈
    x = inception_module(x,
                         filters_1x1=256,
                         filters_3x3_reduce=160,
                         filters_3x3=320,
                         filters_5x5_reduce=32,
                         filters_5x5=128,
                         filters_pool_reduce=128,
                         name='inception_4e')

    x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3_2')(x)
    # 여덟번째 inception 모듈
    x = inception_module(x,
                         filters_1x1=256,
                         filters_3x3_reduce=160,
                         filters_3x3=320,
                         filters_5x5_reduce=32,
                         filters_5x5=128,
                         filters_pool_reduce=128,
                         name='inception_5a')
    # 아홉번째 inception 모듈
    x = inception_module(x,
                         filters_1x1=384,
                         filters_3x3_reduce=192,
                         filters_3x3=384,
                         filters_5x5_reduce=48,
                         filters_5x5=128,
                         filters_pool_reduce=128,
                         name='inception_5b')

    x = GlobalAveragePooling2D(name='avg_pool_5_3x3_1')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_classes, activation='softmax', name='output')(x)
    
    model = Model(inputs=input_tensor, outputs=output)
    model.summary()
    
    return model

In [3]:
model = googlenet((224, 224, 3), n_classes=10)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_1_7x7_2 (Conv2D)         │ (None, 112, 112, 64)      │           9,472 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pool_1_3x3_2              │ (None, 56, 56, 64)        │               0 │ conv_1_7x7_2[0][0]         │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_2a_3x3_1 (Conv2D)        │ (None, 56, 56, 64)        │           4,160 │ max_pool_1_3x3_2[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_2b_3x3_1 (Conv2D)        │ (None, 56, 56, 192)       │         110,784 │ conv_2a_3x3_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pool_2_3x3_2              │ (None, 28, 28, 192)       │               0 │ conv_2b_3x3_1[0][0]        │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 28, 28, 96)        │          18,528 │ max_pool_2_3x3_2[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 28, 28, 16)        │           3,088 │ max_pool_2_3x3_2[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 28, 28, 192)       │               0 │ max_pool_2_3x3_2[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 28, 28, 64)        │          12,352 │ max_pool_2_3x3_2[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 28, 28, 128)       │         110,720 │ conv2d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 28, 28, 32)        │          12,832 │ conv2d_3[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 28, 28, 32)        │           6,176 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ inception_3a (Concatenate)    │ (None, 28, 28, 256)       │               0 │ conv2d[0][0],              │
│                               │                           │                 │ conv2d_2[0][0],            │
│                               │                           │                 │ conv2d_4[0][0],            │
│                               │                           │                 │ conv2d_5[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_7 (Conv2D)             │ (None, 28, 28, 128)       │          32,89

 Total params: 5,983,802 (22.83 MB)

 Trainable params: 5,983,802 (22.83 MB)

 Non-trainable params: 0 (0.00 B)